In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

# Project imports
from src.data_loader import load_and_filter_data
from src.preprocessing import split_features_labels
from src.feature_selection import select_top_features
from src.bayes_optimization import run_all_bayesian_optimizations
from src.optuna_optimization import optimize_with_optuna
from src.xai_interpretation import (
    explain_with_shap,
    explain_with_lime,
    plot_permutation_importance,
    plot_model_feature_importance,
    plot_elasticnet_weights
)

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

model_map = {
    "RandomForest": RandomForestClassifier,
    "CatBoost": CatBoostClassifier,
    "LGBM": LGBMClassifier,
    "XGB": XGBClassifier,
}

# ========================================
# 1. Load + preprocess data
# ========================================
data = load_and_filter_data()
X, y = split_features_labels(data)

# ========================================
# 2. Feature selection
# ========================================
X = X.copy()
selected_features = select_top_features(X, y, top_k=20)
X_selected = X[selected_features]

# ========================================
# 3. Run both optimization methods
# ========================================
print("🔍 Running Bayesian Optimization...")
bayes_results = run_all_bayesian_optimizations(X_selected, y)
print("✅ Done.\n")

print("🔍 Running Optuna Optimization...")
optuna_results = optimize_with_optuna(X_selected, y, n_trials=30)
print("✅ Done.\n")

combined = pd.concat([bayes_results, optuna_results], ignore_index=True)

# ========================================
# 4. Find best model across all results
# ========================================
best_model_info = combined.loc[combined['Best CV Score (ROC AUC)'].idxmax()]
best_model_name = best_model_info['Model']
best_params = best_model_info['Best Parameters']

# Cast types
if best_model_name == "RandomForest":
    best_params["n_estimators"] = int(best_params["n_estimators"])
    if isinstance(best_params.get("max_features", 1), float) and best_params["max_features"] > 1:
        best_params["max_features"] = float(best_params["max_features"])
elif best_model_name == "CatBoost":
    best_params["iterations"] = int(best_params["iterations"])
    best_params["depth"] = int(best_params["depth"])
elif best_model_name == "LGBM":
    best_params["n_estimators"] = int(best_params["n_estimators"])
elif best_model_name == "XGB":
    best_params["n_estimators"] = int(best_params["n_estimators"])
    best_params["max_depth"] = int(best_params["max_depth"])

print(f"🏆 Best Model: {best_model_name}")
print("🔧 Best Parameters:", best_params)

# ========================================
# 5. Cross-validated training & evaluation
# ========================================
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
auc_scores = []
final_model = model_map[best_model_name](**best_params)

for train_idx, test_idx in kf.split(X_selected, y):
    X_train, X_test = X_selected.iloc[train_idx], X_selected.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    final_model.fit(X_train, y_train)
    y_pred = final_model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)
    auc_scores.append(auc)

print(f"\n🎯 Mean ROC AUC (10-fold): {np.mean(auc_scores):.4f} ± {np.std(auc_scores):.4f}")



import os

# Create results directory
os.makedirs("results", exist_ok=True)

# Save selected features
with open("results/selected_features.txt", "w") as f:
    f.write("\n".join(selected_features))

# Save optimization results
bayes_results.to_csv("results/bayes_results.csv", index=False)
optuna_results.to_csv("results/optuna_results.csv", index=False)
combined.to_csv("results/final_summary.csv", index=False)

# Save final cross-val AUC
with open("results/final_auc.txt", "w") as f:
    f.write(f"Mean AUC: {np.mean(auc_scores):.4f}\n")
    f.write(f"Std AUC: {np.std(auc_scores):.4f}\n")
    f.write(f"Model: {best_model_name}\n")
    f.write(f"Params: {best_params}\n")


/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🔍 Running Bayesian Optimization...
|   iter    |  target   | n_esti... | max_fe... |
-------------------------------------------------
| 1         | 0.7916363 | 106.18101 | 0.9556428 |
| 2         | 0.8119999 | 159.79909 | 0.6387926 |
| 3         | 0.8142727 | 73.402796 | 0.2403950 |
| 4         | 0.7962727 | 58.712541 | 0.8795585 |
| 5         | 0.7927272 | 140.16725 | 0.7372653 |
| 6         | 0.8036363 | 53.087674 | 0.9729188 |
| 7         | 0.8332727 | 174.86639 | 0.2911051 |
| 8         | 0.8248181 | 77.273745 | 0.2650640 |
| 9         | 0.8034545 | 95.636336 | 0.5722807 |
| 10        | 0.8028181 | 114.79175 | 0.3621062 |
| 11        | 0.7814545 | 182.06330 | 0.9650160 |
| 12        | 0.8219999 | 172.88162 | 0.1216336 |
| 13        | 0.7916363 | 176.62830 | 1.0       |
| 14        | 0.8158181 | 82.794918 | 0.4234503 |
| 15        | 0.8332727 | 174.87816 | 0.2782414 |
| 16        | 0.7762727 | 75.799788 | 0.8199511 |
| 17        | 0.8230909 | 174.08304 | 0.1       |
| 18        | 0

[I 2025-07-29 11:47:27,469] A new study created in memory with name: no-name-a91a173d-144d-44ce-b4bc-5ecafaff406c


| 40        | 0.8110909 | 179.52723 | 0.1610677 | 4.5302988 |
✅ Done.

🔍 Running Optuna Optimization...
🔍 Optimizing RandomForest with Optuna...


[I 2025-07-29 11:47:31,986] Trial 0 finished with value: 0.844 and parameters: {'n_estimators': 94, 'max_depth': 44, 'min_samples_split': 4, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.844.
[I 2025-07-29 11:47:32,307] Trial 1 finished with value: 0.8341818181818181 and parameters: {'n_estimators': 55, 'max_depth': 46, 'min_samples_split': 2, 'min_samples_leaf': 7}. Best is trial 0 with value: 0.844.
[I 2025-07-29 11:47:32,992] Trial 2 finished with value: 0.8343636363636364 and parameters: {'n_estimators': 138, 'max_depth': 35, 'min_samples_split': 9, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.844.
[I 2025-07-29 11:47:33,770] Trial 3 finished with value: 0.8298181818181819 and parameters: {'n_estimators': 145, 'max_depth': 30, 'min_samples_split': 7, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.844.
[I 2025-07-29 11:47:34,564] Trial 4 finished with value: 0.8123636363636363 and parameters: {'n_estimators': 164, 'max_depth': 7, 'min_samples_split': 10, 'min

...

75:	learn: 0.0134280	total: 190ms	remaining: 65.1ms
76:	learn: 0.0132082	total: 192ms	remaining: 62.3ms
77:	learn: 0.0129478	total: 193ms	remaining: 59.5ms
78:	learn: 0.0126995	total: 195ms	remaining: 56.8ms
79:	learn: 0.0124637	total: 197ms	remaining: 54ms
80:	learn: 0.0122330	total: 198ms	remaining: 51.3ms
81:	learn: 0.0119616	total: 200ms	remaining: 48.7ms
82:	learn: 0.0117955	total: 202ms	remaining: 46.2ms
83:	learn: 0.0116429	total: 203ms	remaining: 43.6ms
84:	learn: 0.0114046	total: 205ms	remaining: 41ms
85:	learn: 0.0111945	total: 206ms	remaining: 38.4ms
86:	learn: 0.0109864	total: 208ms	remaining: 35.9ms
87:	learn: 0.0107731	total: 210ms	remaining: 33.3ms
88:	learn: 0.0106356	total: 211ms	remaining: 30.8ms
89:	learn: 0.0104745	total: 213ms	remaining: 28.3ms
90:	learn: 0.0103540	total: 214ms	remaining: 25.9ms
91:	learn: 0.0102310	total: 216ms	remaining: 23.4ms
92:	learn: 0.0101130	total: 217ms	remaining: 21ms
93:	learn: 0.0099969	total: 219ms	remaining: 18.6ms
94:	learn: 0.00982

[I 2025-07-29 11:50:13,488] Trial 28 finished with value: 0.8054545454545454 and parameters: {'iterations': 70, 'depth': 7, 'learning_rate': 0.10224770110784631}. Best is trial 25 with value: 0.8361818181818181.


8:	learn: 0.3335077	total: 108ms	remaining: 732ms
9:	learn: 0.3059308	total: 118ms	remaining: 711ms
10:	learn: 0.2867887	total: 130ms	remaining: 698ms
11:	learn: 0.2699781	total: 132ms	remaining: 639ms
12:	learn: 0.2544177	total: 137ms	remaining: 601ms
13:	learn: 0.2386503	total: 139ms	remaining: 557ms
14:	learn: 0.2253376	total: 142ms	remaining: 520ms
15:	learn: 0.2150123	total: 144ms	remaining: 485ms
16:	learn: 0.2034670	total: 146ms	remaining: 455ms
17:	learn: 0.1917223	total: 148ms	remaining: 429ms
18:	learn: 0.1822609	total: 150ms	remaining: 404ms
19:	learn: 0.1735387	total: 156ms	remaining: 391ms
20:	learn: 0.1648256	total: 160ms	remaining: 372ms
21:	learn: 0.1571747	total: 163ms	remaining: 356ms
22:	learn: 0.1483763	total: 166ms	remaining: 339ms
23:	learn: 0.1408763	total: 168ms	remaining: 322ms
24:	learn: 0.1346689	total: 170ms	remaining: 307ms
25:	learn: 0.1278889	total: 173ms	remaining: 292ms
26:	learn: 0.1224731	total: 175ms	remaining: 278ms
27:	learn: 0.1178335	total: 177ms

[I 2025-07-29 11:50:15,654] Trial 29 finished with value: 0.8110909090909091 and parameters: {'iterations': 83, 'depth': 8, 'learning_rate': 0.053784467754643336}. Best is trial 25 with value: 0.8361818181818181.


66:	learn: 0.0872863	total: 408ms	remaining: 97.4ms
67:	learn: 0.0856858	total: 411ms	remaining: 90.7ms
68:	learn: 0.0841042	total: 415ms	remaining: 84.2ms
69:	learn: 0.0826102	total: 420ms	remaining: 77.9ms
70:	learn: 0.0811110	total: 424ms	remaining: 71.6ms
71:	learn: 0.0797879	total: 427ms	remaining: 65.3ms
72:	learn: 0.0785342	total: 431ms	remaining: 59ms
73:	learn: 0.0773876	total: 435ms	remaining: 52.9ms
74:	learn: 0.0763917	total: 438ms	remaining: 46.8ms
75:	learn: 0.0750616	total: 442ms	remaining: 40.7ms
76:	learn: 0.0735405	total: 446ms	remaining: 34.7ms
77:	learn: 0.0725287	total: 449ms	remaining: 28.8ms
78:	learn: 0.0714280	total: 453ms	remaining: 22.9ms
79:	learn: 0.0700779	total: 457ms	remaining: 17.1ms
80:	learn: 0.0691570	total: 460ms	remaining: 11.4ms
81:	learn: 0.0681695	total: 464ms	remaining: 5.66ms
82:	learn: 0.0669912	total: 468ms	remaining: 0us
0:	learn: 0.6591998	total: 4.61ms	remaining: 332ms
1:	learn: 0.6133472	total: 8.59ms	remaining: 305ms
2:	learn: 0.5855733

In [2]:
print("\n📊 ElasticNet Feature Weights:")
plot_elasticnet_weights(X_selected, y)


📊 ElasticNet Feature Weights:


/workspaces/NeuroML-PTSD/src/xai_interpretation.py:95: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=feature_names, y=sorted_coef, palette=colors)


In [38]:
import shap

final_model.fit(X_selected, y)
# Explain the model's predictions using SHAP
explainer = shap.TreeExplainer(final_model)
shap_values = explainer.shap_values(X_selected)


In [39]:
shap.summary_plot(shap_values[:, :, 1], X_selected)

In [ ]:
shap.dependence_plot('IQ', shap_values[:, :, 1], X_selected)

IndexError: index 20 is out of bounds for axis 0 with size 20

In [40]:
import matplotlib.pyplot as plt

plt.figure()
shap.summary_plot(shap_values[:, :, 1], X_selected, show=False)
plt.savefig("results/shap_summary_plot.png", bbox_inches="tight")
plt.close()